In [ ]:
import logging
from datetime import datetime

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
loger = logging.getLogger("agentcore-memory")

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "FitnessCoach"
memory_id = None

In [ ]:
try:
    print("Creating Memory...")
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Fitness Coach Agent",
        strategies=[],
        event_expiry_days=7,
        max_wait=300,
        pool_interval=10
    )

    memory_id = memory['id']
    logger.info(f"Memory created sucessfully with id: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and 'already exists' in str(e):
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory id: {memory_id}")
except Exception as e:
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to cleanup memory: {cleanup_error}")

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HuamnMessage, SystemMessage
from langchain_aws import ChatBedrock

In [ ]:
def create_agent(client, memory_id, actor_id, session_id):
    """Create and configure the LangGraph agent"""

    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  
        model_kwargs={"temperature": 0.1}
    )

    @tool
    def list_events():
        events = client.list_events(
            memory_id=memory_id,
            actor_id=actor_id,
            session_id=session_id,
            max_results=10
        )
        return events

    tools = [list_events]
    llm_with_tools = llm.bind_tools(tools)

    system_message = """
    You are the personal fitness coach, a sophisticated fitness guaidance assistant.
    PURPOSE:
    - Help users develop workout routines based on their fitness goals
    - Remember user's exercises preferences, limitations and progress
    - Provide personalized fitness recommendations and training plans
    MEMORY CAPABILITIES:
    - You have access to recent events with the list_events tool
    """

    def chatbot(state: MessagesState):
        raw_messages = state["messages"]

        non_system_messages = [msg for msg in raw_messages if not isinstance[msg, SystemMessage]]

        messages = [SystemMessages(content=system_message)] + non_system_messages

        latest_user_message = next((msg.content for msg in reversed(messages) if isinstance(msg, HumanMessage)), None)

        response = llm_with_tools.invoke(messages)

        if latest_user_message and response.content.strip():
            conversation = [
                (latest_user_message, "USER"),
                (response.content, "ASSISTANT")
            ]

            if all(msg[0].strip() for msg in conversation):  
                try:
                    client.create_event(
                        memory_id=memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=conversation
                    )
                except Exception as e:
                    print(f"Error saving conversation: {str(e)}")

        return {"messages": raw_messages + [response]}

    graph_builder = StateGraph(MessagesState)

    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))

    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition
    )
    graph_builder.add_edge("tools", "chatbot")

    graph_builder.set_entry_point("chatbot")

    return graph_builder.compile()

In [ ]:
def langgraph_bedrock(paylod, agent):
    user_input = payload.get("prompt")

    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    return response["messages"][-1]

In [ ]:
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"workout-{datetime.now().strftime('%Y%m%d%H%M%S')}"

In [ ]:
agent = create_agent(client, memory_id, actor_id, session_id)

In [ ]:
response = langgraph_bedrock({"prompt": "Hello! This is my first day. I need a workout routine!"}, agent)
print(f"Agent: {response}")

In [ ]:
new_agent = create_agent(client. memory_id, actor_id, session_id)

response = langgraph_bedrock(
    {
        "prompt": "Hello again! Can you remaind me about my last workout session?"
    }, new_agent
)
print("New Agent Session:\n")
print(f"Agent: {response}")